<a href="https://colab.research.google.com/github/Feindrial/bondnet/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
from tensorflow import keras
from pathlib import Path

In [ ]:
print(tf.__version__)

locc = "COLAB"

if locc == "LOCAL":

    #veri setinin indirilip dosya yollarının kaydedilmesi

    movielens_url = "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

    downloaded_path = tf.keras.utils.get_file(movielens_url.rsplit('/').pop(), movielens_url, extract=True)

    dataset_dir = Path(downloaded_path).parent / Path(downloaded_path).stem
    print(dataset_dir)

    #indirelen veri setinin ilgili csv dosyaları pandas dataframe olarak atandı

elif locc == "COLAB":

    dataset_dir = Path("/content")
    print(dataset_dir)

    ! pip install -q kaggle
    from google.colab import files
    files.upload()

    ! mkdir ~/.kaggle
    ! mv kaggle.json ~/.kaggle/
    ! chmod 600 ~/.kaggle/kaggle.json

    !kaggle datasets download "shubhammehta21/movie-lens-small-latest-dataset"
    !unzip -j "movie-lens-small-latest-dataset"

else:
    raise RuntimeError

2.12.0
/content


In [2]:
ratings_file = dataset_dir / "ratings.csv"
ratings = pd.read_csv(
    ratings_file
)

movies_file = dataset_dir / "movies.csv"
movies = pd.read_csv(
    movies_file
)

#TODO doğal dil işleme ile kullanıcıların ilgili filme yaptığı yorumlar çıkarılabilir

# tags_file = dataset_dir / "tags.dat"
# tags = pd.read_csv(tags_file)

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [ ]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [3]:
#filmlerin kaç farklı türü olduğu bulundu ve bu bulunan türler "movies" dataframe'ine sütun olarak eklendi
unique_genres = pd.unique(movies["genres"].str.split("|", expand=True).values.ravel())
movies[unique_genres] = 0

#hafızada daha az yer kaplaması açısından bu film türlerini tutan sütunlar 1 byte olarak tutuldu
movies = movies.astype({x: np.int8 for x in unique_genres})

#bir yukarıdaki hücrede de görüleceği üzere indirilen dataframe'de film türleri tek sütünda ve "|" sembolü ile ayrılmış durumda
#üstteki 3 satırlık kodda bu türleri kendi ayrı sütunlarına almıştık 
#alttaki for döngüsünde ise ilgili filme denk gelen tür sütunları eğer bu filmin türünü barındırıyor ise 1 yapıldı
#eğer barındırmıyor ise üstteki ilgili koddan dolayı 0 olarak kaldı
#örneğin "3" numaralı filmin türleri "Comedy|Romance" olduğu için
#movies dataframe'inin "Comedy" ve "Romance" sütununun değerleri "3" numaralı film için 1 oldu
for index, row in enumerate(movies["genres"].str.split("|")):
    movies.loc[index, row] = 1

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             9742 non-null   int64 
 1   title               9742 non-null   object
 2   genres              9742 non-null   object
 3   Adventure           9742 non-null   int8  
 4   Animation           9742 non-null   int8  
 5   Children            9742 non-null   int8  
 6   Comedy              9742 non-null   int8  
 7   Fantasy             9742 non-null   int8  
 8   None                9742 non-null   int8  
 9   Romance             9742 non-null   int8  
 10  Drama               9742 non-null   int8  
 11  Action              9742 non-null   int8  
 12  Crime               9742 non-null   int8  
 13  Thriller            9742 non-null   int8  
 14  Horror              9742 non-null   int8  
 15  Mystery             9742 non-null   int8  
 16  Sci-Fi              9742

In [4]:
#bir yukarıdaki hücrede bu türleri filmlere göre haritaladığımız için
#dataframe'de bulunan "genres" ve "title" sütunlarını hafızadan kazanmak için sildik
#böylece elimizde sadece ilgili filmin "id"si ve türleri kaldı
movies = movies.drop(["genres", "title"], axis=1)
movies.head(10)

,movieId,Adventure,Animation,Children,Comedy,Fantasy,None,Romance,Drama,Action,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,10,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#rating dataframe'inde ilgili kullanıcının ilgili filme kaç puan verdiği bulunuyor ama bu filmin hangi türlere sahip olduğu bulunmuyor
#bu yüzden yukarıdaki hücrelerde oluşturduğumuz "movies" dataframe'indeki ilgili "id"
#ratings dataframe'indeki "movieId"ler ile haritalandı
ratings = ratings.join(movies.set_index("movieId"), on="movieId", how="inner")

In [6]:
#kullanılmadığı için "timestamp" sütunu dataframe'den silindi ve
#kullanılacak modelin veriyi ezberlemesinden kaçınmak için dataframe'in satırları rastgele olarak karıştırıldı
ratings = ratings.drop("timestamp", axis=1)
ratings = ratings.sample(frac=1, random_state=1337)
ratings.head(10)

,userId,movieId,rating,Adventure,Animation,Children,Comedy,Fantasy,None,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
74273,474,4356,3.0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
73740,474,2203,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12593,80,6281,3.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30714,215,55820,4.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84194,537,80906,5.0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
48801,317,110,5.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
16319,105,2931,4.5,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
73716,474,2144,3.0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
98035,606,4881,4.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1445,15,356,5.0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 74273 to 14886
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   userId              100836 non-null  int64  
 1   movieId             100836 non-null  int64  
 2   rating              100836 non-null  float64
 3   Adventure           100836 non-null  int8   
 4   Animation           100836 non-null  int8   
 5   Children            100836 non-null  int8   
 6   Comedy              100836 non-null  int8   
 7   Fantasy             100836 non-null  int8   
 8   None                100836 non-null  int8   
 9   Romance             100836 non-null  int8   
 10  Drama               100836 non-null  int8   
 11  Action              100836 non-null  int8   
 12  Crime               100836 non-null  int8   
 13  Thriller            100836 non-null  int8   
 14  Horror              100836 non-null  int8   
 15  Mystery             100836 non-

In [7]:
#daha sonra kullanmak adına tekil kullanıcı sayısı ve film sayısı bulundu
num_unique_users = len(ratings["userId"].unique())
num_unique_movies = len(ratings["movieId"].unique())

#daha sonra kullanmak adına herhangi bir kullanıcı tarafından verilen en düşük ve en yüksek puan bulundu
min_rating = ratings["rating"].min()
max_rating = ratings["rating"].max()

#modele giriş olarak verilecek değerler x_train değişkeninde toplandı 
#bunlar kullanıcı, film "id"leri ve ilgili filmin türleri
x_train = ratings.drop("rating", axis=1).values

#filmlere kullanıcılar tarafından verilen puanlar reel sayılar kümesinde tanımlı olmadığı için
#örneğin 3.123 veya 1.567 olmadığı; 0.5, 2.5, 4.0 gibi belirli bir oranda artan sonlu sayılar kümesini oluşturduğu için
#bu değerler kategorisel olarak tutuldu
#örneğin bir kullanıcı bir filme 2.5 verdiyse min_rating değişkeninde başlamak ve max_rating değişkeninde bitmek üzere
#ilgili oranda artan(bu durumda 0.5) bir kategori kümesi oluşturuldu
#bu kategori kümesinde 2.5 puanı hangi kategori indisine denk geliyorsa ise o kategori indisinin değeri 1 yapıldı
y_labels = ratings["rating"]

#ilginç bir hata yüzünden
from sklearn.preprocessing import LabelEncoder
y_train = keras.utils.to_categorical(LabelEncoder().fit_transform(y_labels))

# y_train = keras.utils.to_categorical(sk.preprocessing.LabelEncoder().fit_transform(y_labels))

In [ ]:
#burada yukarıdaki hücrede x_train değişkeninde tutulan giriş verilerini görsel olarak görebiliyoruz
#ilk iki eleman kullanıcı, film "id"leri olmak üzere geri kalan değerler ise ilgili filmin
#sahip olduğu(bu durumda 1 değerini alan) ve/veya olmadığı(bu durumda 0 değerini alan) değerleri tutan türleri görüyoruz
x_train[:3]

array([[ 474, 4356,    0,    0,    0,    1,    0,    0,    1,    0,    0,
           0,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,
           0],
       [ 474, 2203,    0,    0,    0,    0,    0,    0,    0,    1,    0,
           1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [  80, 6281,    0,    0,    0,    0,    0,    0,    0,    1,    0,
           0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]])

In [ ]:
#bu hücrede yukarıda anlatılmaya çalışılan mantık görsel olarak görülüyor
#bu dizaynın uygulanma nedeni ise oluşturulan modelin ilgili reel sayı aralığında tahmin yapması yerine
#ilgili puan kategorilerinden(0.5, 2.0, 4.5 gibi) hangisi olduğunu tahmin etmesi yeterli hale gelecek ve
#daha doğru bir çıktı elde edilecek olmasıdır
y_train[:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
ratings["rating"].unique()

array([3. , 4. , 3.5, 5. , 4.5, 1.5, 0.5, 2. , 1. , 2.5])

In [1]:
!pip install surprise

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

# Load the MovieLens 100K dataset
data = Dataset.load_builtin('ml-100k')

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Define the embedding dimensions and regularization parameters
num_users = trainset.n_users
num_items = trainset.n_items
num_factors = 50
reg_user = 0.01
reg_item = 0.01

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095416 sha256=1dfeaab8f919b08bf9f27d44c96049e2f172e0c2e58528bebd12cfac63737239
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [3]:
print(trainset.)

In [ ]:
del ratings
del movies
# del tags
del y_labels

import gc
gc.collect()

63

In [8]:
class DataInfo:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def get_slice(self):
        return slice(self.start, self.end)

    def get_size(self):
        return self.end - self.start


class DataInfoIter:
    def __init__(self, data, step=1):
        self.step = step
        self.data = data
        self.end = data.start
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.end >= self.data.end:
            raise StopIteration

        start = self.end

        if self.end + self.step >= self.data.end:
            self.end = self.data.end
        else:
            self.end += self.step
        
        return DataInfo(start, self.end)

        

#TODO sınıf farklı veri setlerine göre kendini uyumlu hale getiremiyor, daha dinamik bir mimari uygulanması gerekli  
class BondNet(keras.Model):
    def __init__(self, features: np.array, labels: np.array, num_unique_users, num_unique_movies, min_rating, max_rating,
                 rating_step=0.5, embedding_size=32, batch_size=32, train_split=0.6, val_split=0.2,
                 num_factors=100, reg_user=0.01, reg_item=0.01, **kwds):         
        '''bu fonksiyon katmanların ve diğer parameterlerin oluşturulduğu fonksiyondur'''
        


        if (train_split + val_split >= 1.0) or (len(features) != len(labels)):
            raise RuntimeError


        
        def ContiguousDataInfo(splits, start=0):
            data = []

            st = start
            end = start
            for sp in splits:
                end += sp
                data.append(DataInfo(st, end))
                st += sp

            return data




        super(BondNet, self).__init__(**kwds)
        
        ##TODO veriyi ayırma statik olarak belirlenmemeli, train fonksiyonunda parametre olarak alınmalı  


        __perform_SVD()

        
        self.train_split, self.val_split, self.test_split = ContiguousDataInfo((int(train_split * len(features)), int(val_split * len(features)), int((1.0 - (train_split + val_split)) * len(features))))
        print(self.train_split.get_slice(), self.val_split.get_slice(), self.test_split.get_slice())
        print(int(train_split * len(features)), int((train_split + val_split) * len(features)), len(features))
        # self.train_split = (0, int(train_split * len(features)))
        # self.val_split = (int(train_split * len(features)), int(train_split * len(features)) + int(val_split * len(features)))
        # self.test_split = (int(train_split * len(features)) + int(val_split * len(features)), len(features))

        self.features = features
        self.labels = labels

        self.id_input, self.mfeature_input = ContiguousDataInfo((2, len(self.features[0, 2:])))

        #kullanıcılar ve filmler için kullanılacak embedding vektörlerinin uzunluğunun ne kadar olacağının tutulduğu değişken
        # self.id_input_slice = slice(0, 2)
        # self.mfeature_input_slice = slice(2, len(self.features[0, 2:]))
        self.boutput = DataInfo(0, int(((max_rating - min_rating) / rating_step) + 1))
        self.embedding_size = embedding_size
        self.batch_size = batch_size

        #EMBEDDING API
        
        #burada hashing yaparak ilgili kullanıcı ve filmin aşağıda görülen embedding katmanında
        #hangi satıra denk geldiğini haritalandı
        #örneğin "42" numaralı kullanıcının embedding vektörü embedding katmanında "35" numaralı indise(satıra) denk gelebilir 
        #çünkü kullanıcı veya film "id"leri 0'dan başlayarak birer birer artmak zorunda değil veya
        #integer bir değer olmak zorunda bile değil
        #bu yüzden bu değerleri ilgili kullanıcı veya film embedding vektörüne denk gelen satırın indisine göre haritalamamız gerekli
        self.user_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(np.unique(features[:, 0]), np.arange(num_unique_users)),
            default_value=-1
        )
        self.movie_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(np.unique(features[:, 1]), np.arange(num_unique_movies)),
            default_value=-1
        )

        #bu embedding vektörleri için alınan girişi barındıran katman
        #yani bu katman ile sadece "userId" ve "movieId" değerlerini alıyoruz
        #çünkü sadece bunlar embedding katmanları için kullanılacak, film türleri için farklı bir API kullanacağız
        self.embedding_input = keras.layers.InputLayer(
            input_shape=(self.id_input.get_size(),),
            dtype=np.int64
        )

        #burada collaborative filtering modelinde kullanılan matrix factorization metodunu kullanmak adına
        #her bir "tekil" kullanıcı ve filme "embedding_size" boyutunda bir embedding vektörü ve 
        #her bir kullanıcı ve film özünde tekil kavramlar olduğu için birer integer sapma değeri(bias) tahsis edildi böylece
        #vektörler üzerinde dot product işlemi kullanılarak ve daha sonra sapma değerleri ile toplanarak ilgili puan elde edilebilir 
        #daha sonra da modelin çıktısında tahmin edilen kullanıcının ilgili filme verdiği puan kategorisini
        #gerçekte olan kategori ile karşılaştırarak ilgili embedding vektörlerini 
        #daha doğru sonucu verdiği değerlere doğru değiştirebiliriz
        #yani ilgili loss değerine göre ilgili vektör ağırlıklarını(weights)
        #daha doğru sonucu verdiği yönde(gradient descent işlemi) değiştirilebilir
        self.user_embedding = keras.layers.Embedding(
            input_dim=num_unique_users,
            output_dim=embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.L1(l1=3e-3)
        )
        self.user_bias = keras.layers.Embedding(
            input_dim=num_unique_users,
            output_dim=1
        )

        self.movie_embedding = keras.layers.Embedding(
            input_dim=num_unique_movies,
            output_dim=embedding_size,
            embeddings_initializer="he_normal",  
            embeddings_regularizer=keras.regularizers.L1(l1=3e-3)
        )
        self.movie_bias = keras.layers.Embedding(
            input_dim=num_unique_movies,
            output_dim=1
        )

        #VISIBLE FEATURES API

        #bu katman ise film türlerini giriş olarak alan katman 
        #burada giriş olarak verilen film türleri ilgili gizli katmanlardan geçtikten sonra(şuan için 3 tane dense layer var)
        #iki boyuttan tek boyuta indirildiği(flatten layer ile) ve daha sonra kullanmak için hazır hale getirildiği katman
        #bu API'ın görevi "call" fonksiyonunda anlatılmıştır
        self.visible_input = keras.layers.InputLayer(
            input_shape=(self.mfeature_input.get_size(),),
            dtype=np.int8
        )
        self.dense1 = keras.layers.Dense(
            self.embedding_size * 16,
            activation="relu"
        )
        self.dense2 = keras.layers.Dense(
            self.embedding_size * 8,
            activation="relu"
        )
        self.dense3 = keras.layers.Dense(
            self.embedding_size * 4,
            activation="relu"
        )
        self.vis_drop = keras.layers.Dropout(0.2)
        self.visible_flatten = keras.layers.Flatten()

        #en önemli kısımlardan biri olan modelin çıkışını aldığımız katmanda ise kullanıcının ilgili filme verdiği puanın
        #hangi kategoriye ait olduğunun tahmini yapılıyor
        #örneğin en düşük puanın(min_rating) 0.0, en yüksek puanın(max_rating) 5.0 ve puan artım oranın(rating_step) 1.0 olduğu senaryoda
        #((5.0 - 0.0) / 1.0) + 1) = 6 olmak üzere toplam 6 tane kategori barındırdığı(bu kategoriler 0.0, 1.0, 2.0, 3.0, 4.0, 5.0)
        #ve çıktının da bu kategorilerden biri olarak seçildiği katmanın olduğu kısım bu çıktı katmanıdır
        #bu örneğe itafen eğer çıktı(çıktılar 6 elemanlı diziler olacak çünkü toplamda 6 kategori var)
        #[0, 0, 1, 0, 0, 0] ise seçilen kategori, kullanıcının ilgili filme "2.0" puanını verdiğini 
        #tahmin eden değeri temsil eden kategoridir
        #[0, 0, 0, 0, 0, 1] ise seçilen kategori, max_rating değeri yani "5.0"dır
        self.output_labels = keras.layers.Dense(
            self.boutput.get_size(),
            activation="softmax"
        )

    def call(self, inputs):
        '''bu fonksiyon girişlerin modelin ilgili katmanlarına iletilmesini sağlayan fonksiyondur'''
        '''bu fonksiyonda bulunan "inputs" parametresi x_train değişkenin satırlarını barındırır'''
        '''yani kullanıcı, film "id"leri ve filmlerin türleri'''

        #x_train değişkeninde yer alan ilk iki eleman sırası ile "userId" ve "movieId" olduğu için bu "id"lere denk gelen
        #embedding vektörlerini bulmak adına hash tablosunda ilgili satırların indisi bulunur 
        user_index = self.user_table.lookup(inputs[0][:, 0])
        movie_index = self.movie_table.lookup(inputs[0][:, 1])

        #daha sonra ilgili kullanıcılara ve filmlere denk gelen embedding vektörleri ve sapma değerleri(bias)
        #embedding ve sapma değerleri(bias) katmanından çekilir 
        user_vector = self.user_embedding(user_index)
        user_bias = self.user_bias(user_index)

        movie_vector = self.movie_embedding(movie_index)
        movie_bias = self.movie_bias(movie_index)

        #matrix factorization metodunda kullanılan dot product işlemi 
        #ilgili embedding vektörleri üzerinde uygulanır ve sapma değerleri ile toplanarak ilgili değerler elde edilir
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        embed_out = dot_user_movie + user_bias + movie_bias

        #hatırlanacağı üzere x_train değişkeninde 2. elemandan sonrası film türleri tutuyordu
        #bu film türleri "__init__" fonksiyonunda belirtilen gizli katmanlara giriş olarak verilerek
        #türlerin kodlandığı(encoding) değerler hesaplanır
        genres_vector = self.vis_drop(self.dense3(self.dense2(self.dense1(self.visible_input(inputs[1])))))
        genres_out = self.visible_flatten(genres_vector)             

        #bu embedding değerleri ile tür değerleri birleştirelerek çıkış katmanına giriş olarak verilmeye hazır hale getirilir
        out = tf.concat([embed_out, genres_out], axis=1)

        #ardından son katman olan sınıflandırmanın yapıldığı katmanda ise ilgili kullanıcıların ilgili filmlere 
        #hangi puanı verdiğinin kategorisinin tahmininin yapılması sağlanır
        #daha sonra da, bahseldiği üzere tahmin edilen değer kategorisi ile gerçek değerin kategorisi karşılaştıralarak 
        #ilgili embedding vektörlerinin değerlerinin ve 
        #ilgili filmlerin türlerinin kodlarının(encoding) elde edilmesinin sağlayan
        #rgizli katmanlara(dense layers) matematiksel olarak gömülen değerlerin
        #gerçek değeri elde etmeye doğru iyileştirilmesi sağlanır
        return self.output_labels(out)
            
    def __data_gen(self, split, is_test=False):
        iter = DataInfoIter(split, self.batch_size)

        for i in iter:
            yield (np.asarray(self.features[i.get_slice(), self.id_input.get_slice()], dtype=np.int64), \
                   np.asarray(self.features[i.get_slice(), self.mfeature_input.get_slice()], dtype=np.int8)), \
                   np.asarray(self.labels[i.get_slice()], dtype=np.int8)

    def get_dataset(self):
        signature = ((tf.TensorSpec(shape=(None, self.id_input.get_size()), dtype=tf.int64),
                      tf.TensorSpec(shape=(None, self.mfeature_input.get_size()), dtype=tf.int8)),
                      tf.TensorSpec(shape=(None, self.boutput.get_size()), dtype=tf.int8))

        train_dataset = tf.data.Dataset.from_generator(
            lambda: self.__data_gen(self.train_split),
            output_signature=signature
        )

        val_dataset = tf.data.Dataset.from_generator(
            lambda: self.__data_gen(self.val_split),
            output_signature=signature
        )

        test_dataset = tf.data.Dataset.from_generator(
            lambda: self.__data_gen(self.test_split),
            output_signature=signature
        )

        return train_dataset, val_dataset, test_dataset

    #TODO model fonksiyonlarının kendi implementasyon biçimleri ile çağrılması gerek, keras API'i ile değil

    def cmpl():
        pass
    def trn():
        pass

In [9]:
bondnet = BondNet(
    x_train,
    y_train,
    num_unique_users,
    num_unique_movies,
    min_rating,
    max_rating,
    rating_step=0.5,
    embedding_size=64
)

#modelde kullanılacak loss ve optimizer algoritmaları seçilir
bondnet.compile(
    run_eagerly=True,
    optimizer="Adam",
    loss="MeanSquaredLogarithmicError",
    metrics=[
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.RootMeanSquaredError(),
        "accuracy"
    ]
)

slice(0, 60501, None) slice(60501, 80668, None) slice(80668, 100835, None)
60501 80668 100836


In [10]:
tr, val, tst = bondnet.get_dataset()

In [11]:
#modelin eğitim aşaması fonksiyonlardan alınan veri setleri ile başlatılır
history = bondnet.fit(
    tr.repeat(),
    epochs=10,
    batch_size=bondnet.batch_size,
    steps_per_epoch=bondnet.train_split.get_size() // bondnet.batch_size,
    validation_steps=bondnet.val_split.get_size() // bondnet.batch_size,
    validation_data=val.repeat()
)
history.history

Epoch 1/10


1890/1890 [==============================] - 147s 73ms/step - loss: 0.5244 - mean_squared_error: 0.0829 - root_mean_squared_error: 0.2879 - accuracy: 0.2861 - val_loss: 0.3013 - val_mean_squared_error: 0.0817 - val_root_mean_squared_error: 0.2859 - val_accuracy: 0.3104
Epoch 2/10
1890/1890 [==============================] - 124s 65ms/step - loss: 0.2903 - mean_squared_error: 0.0803 - root_mean_squared_error: 0.2834 - accuracy: 0.3220 - val_loss: 0.2941 - val_mean_squared_error: 0.0807 - val_root_mean_squared_error: 0.2841 - val_accuracy: 0.3194
Epoch 3/10
1890/1890 [==============================] - 123s 65ms/step - loss: 0.2895 - mean_squared_error: 0.0792 - root_mean_squared_error: 0.2815 - accuracy: 0.3393 - val_loss: 0.3004 - val_mean_squared_error: 0.0802 - val_root_mean_squared_error: 0.2833 - val_accuracy: 0.3257
Epoch 4/10
1890/1890 [==============================] - 129s 68ms/step - loss: 0.2889 - mean_squared_error: 0.0786 - root_mean_squared_error: 0.2803 - accuracy: 0.3487 

{'loss': [0.5244112014770508,
  0.2902715504169464,
  0.28947219252586365,
  0.2889302372932434,
  0.28851351141929626,
  0.28816092014312744,
  0.2878313958644867,
  0.2875674068927765,
  0.2872958481311798,
  0.28706979751586914],
 'mean_squared_error': [0.08286015689373016,
  0.08032706379890442,
  0.07922244071960449,
  0.07858142256736755,
  0.07810301333665848,
  0.07773783802986145,
  0.07741714268922806,
  0.07717012614011765,
  0.07691019028425217,
  0.07668698579072952],
 'root_mean_squared_error': [0.2878541648387909,
  0.2834204137325287,
  0.28146448731422424,
  0.2803235650062561,
  0.2794690728187561,
  0.27881526947021484,
  0.278239369392395,
  0.2777952253818512,
  0.2773268520832062,
  0.27692413330078125],
 'accuracy': [0.28611111640930176,
  0.3219500780105591,
  0.33931437134742737,
  0.3487076163291931,
  0.35591790080070496,
  0.3614579439163208,
  0.3666672110557556,
  0.3702227473258972,
  0.37311679124832153,
  0.37748268246650696],
 'val_loss': [0.3013277649

In [13]:
eval = bondnet.evaluate(
    tst,
    batch_size=bondnet.batch_size,
    steps=bondnet.test_split.get_size() // bondnet.batch_size
)

630/630 [==============================] - 19s 31ms/step - loss: 0.2917 - mean_squared_error: 0.0797 - root_mean_squared_error: 0.2823 - accuracy: 0.3252


In [14]:
bondnet.summary()

Model: "bond_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 embedding (Embedding)       multiple                  39040     
                                                                 
 embedding_1 (Embedding)     multiple                  610       
                                                                 
 embedding_2 (Embedding)     multiple                  622336    
                                                                 
 embedding_3 (Embedding)     multiple                  9724      
                                                                 
 input_2 (InputLayer)        [(None, 21)]              0         
                                                                 
 dense (Dense)               multiple                  225